In [0]:
import torch
import torch.nn as nn
import os
import numpy as np
import random
from scipy import integrate

In [0]:
torch.autograd.set_detect_anomaly(True)
# Model for Recurrent Marked Temporal Point Process. Based on
# Nan, Du et al Recurrent Marked Temporal Point Processes: Embedding Event
# History to Vector.
# This is a pytorch implementation of his model
class RMTPP(nn.Module):
    # input type_dim: dimension of types, which is a one-hot representation
    # input hidden_dim: dimension of the hidden layer. Default is 1.
    # input n_layers: number of hidden layers. Default is 1.
    # This will initialize the recurrent neural network.
    def __init__(self, type_dim, hidden_dim=1, n_layers=1):
        super(RMTPP,self).__init__()
        self.type_dim = type_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        # linear embedding layer: map from one-hot types to a number.
        self.type_emb = nn.Linear(self.type_dim, 1, bias=True)
        # recurrence layer: use of relu function, and the input are time and type
        self.rnn = nn.RNN(input_size=2, hidden_size=self.hidden_dim, num_layers=self.n_layers,
                          nonlinearity='relu', bias=True, batch_first=True)
        # type generation layer: map from hidden layers to a vector representation of types
        self.type_gen = nn.Linear(self.hidden_dim, self.type_dim, bias=True)
        # time generation layers: map time and hidden layers to generate time
        self.time_linear = nn.Linear(self.hidden_dim+1, 1, bias=True)

    # This is the forward step of neural network
    # Assume both the input has dim(batch, times, features) of dimension 3
    # input whole_time_info: all of the time information
    # input marker_info: training information for marker
    def forward(self, train_time, marker_info):
        marker1 = self.type_emb(marker_info)
        combi_inputs = torch.cat((train_time, marker1), dim=-1)
        out, hidden = self.rnn(combi_inputs)
        out1 = out.contiguous().view(-1, self.hidden_dim)
        type_guess = self.type_gen(out1)
        return type_guess, out

In [0]:
def train(model, device, whole_time_info, event_type, n_features,lr=0.01, n_epochs=1000):
    for parameter in model.parameters():
        parameter.data.fill_(random.uniform(-1,1))
    marker_info, marker_target, whole_marker = type_encode(event_type, n_features)
    train_time, whole_time_info = time_encode(whole_time_info)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(1, n_epochs+1):
        optimizer.zero_grad()
        train_time.to(device)
        marker_info.to(device)
        type_out, hidden_out = model(train_time, marker_info)
        marker_target = torch.reshape(marker_target, (-1,))
        time_diff = whole_time_info[:, 1:, :] - whole_time_info[:, :-1, :]
        combi_input_time = torch.cat((hidden_out, time_diff), dim=-1)
        cif = model.time_linear(combi_input_time)
        hidden_out = torch.reshape(hidden_out, (-1, model.hidden_dim))
        cif = torch.reshape(cif,(-1, 1))
        cif_weight = list(model.time_linear.parameters())
        loss = criterion(type_out, marker_target) + my_loss(cif, cif_weight, hidden_out)
        loss.backward()
        optimizer.step()
        if epoch%100 == 0:
            print("Loss: {}".format(loss.item()))


def my_loss(cif, cif_weight, hidden_out):
    loss = torch.mean(cif+torch.exp(torch.mm(hidden_out,cif_weight[0][:,:-1].transpose(0,1))
                         + cif_weight[1][0])/cif_weight[0][0][-1]-torch.exp(cif)/cif_weight[0][0][-1])

    return -loss


def type_encode(event_type, numb_features):
    event_type_arr = np.array(event_type)
    type_s =np.zeros((len(event_type_arr), len(event_type_arr[0]), numb_features), dtype=np.float32)
    for i in range(len(event_type_arr)):
        for j in range(len(event_type_arr[0])):
            type_s[i][j][event_type_arr[i][j]] = 1
    type_s = torch.from_numpy(type_s)
    return type_s[:,:-1,:], torch.tensor(event_type_arr)[:,1:], type_s


def time_encode(time):
    time_info = np.zeros((len(time), len(time[0]), 1), dtype=np.float32)
    for i in range(len(time)):
        for j in range(len(time[i])):
            time_info[i][j][0] = time[i][j]
    time_for_train = torch.from_numpy(time_info[:, :-1, :])
    whole_time = torch.from_numpy(time_info)
    return time_for_train, whole_time


def make_time_target(time_sample):
    time_target = torch.zeros(len(time_sample), len(time_sample[0]))
    return time_target


def predict(model, device, time, marker, n_features):
    time = [time]
    marker = [marker]
    time_input = time_encode(time)[1]
    type_input = type_encode(marker, n_features)[2]
    type_input.to(device)
    time_input.to(device)
    type_out, hidden_out = model(time_input, type_input)
    type_out = nn.functional.softmax(type_out[-1], dim=0)
    type_int = torch.max(type_out, dim=0)[1].item()
    estimated_time = cal_integral(model, hidden_out[0][-1][0], time_input[0][-1][0])
    return type_int, estimated_time


def cal_integral(model, hidden_out, time):
    hidden_out = hidden_out.item()
    parameters = list(model.time_linear.parameters())
    bias = parameters[1][0].item()
    v = parameters[0][0][0].item()
    w = parameters[0][0][1].item()
    func = lambda x: equation(x, time, hidden_out, bias, v, w)
    y = integrate.quad(func, time, np.inf)
    return y[0]


def equation(time_var, start_time, hidden_out, bias, v, w):
    time_guess = time_var*np.exp(v*hidden_out + w*(time_var-start_time)
                                 + bias+np.exp(v*hidden_out+bias)/w
                                 - np.exp(v*hidden_out + w*(time_var-start_time)+bias)/w)
    return time_guess


def select_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print("You are using GPU acceleration.")
        print("Number of CUDAs(cores): ", torch.cuda.device_count())
    else:
        device = torch.device("cpu")
        print("CUDA is not Available. You are using CPU only.")
        print("Number of cores: ", os.cpu_count())
    return device


def data_process(file_name):
    f = open('time-train.txt','r')
    time_data = []
    file_data = f.readlines()
    f.close()
    for line in file_data:
        data = line.split(" ")
        a_list = []
        for i in range(len(data)):
            if data[i] != "\n":
                a_list.append(float(data[i]))
        time_data.append(a_list)
    return time_data


def generate_type(time_data):
    type_data = []
    for line in time_data:
      new_line = []
      for item in line:
          new_line.append(1)
      type_data.append(new_line)
    return type_data

In [26]:
time_data = data_process("time_train.txt")
type_data = generate_type(time_data)
model = RMTPP(2)
device = select_device()
train(model, device, time_data, type_data, 2)

You are using GPU acceleration.
Number of CUDAs(cores):  1
Loss: 0.261820912361145
Loss: -0.4017449617385864
Loss: -0.5356290340423584
Loss: -0.5663974285125732
Loss: -0.583436131477356
Loss: -0.5932741165161133
Loss: -0.5990686416625977
Loss: -0.6025647521018982
Loss: -0.6048175692558289
Loss: -0.606339693069458


In [31]:
torch.save(model, "model.pt")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type RMTPP. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
